In [1]:
import numpy as np
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from statistics import mode
from Neural_Net import boxplots, dataCleaning
from sklearn.model_selection import train_test_split

In [2]:
dataset_temp = pd.read_csv("LBW_Dataset.csv")
dataset_temp = dataCleaning(dataset_temp)


In [3]:
# features is a dataframe containing all the different features (attributes) of our dataset
features = dataset_temp[['Community','Age','Weight','Delivery phase','HB','IFA','BP','Education','Residence']] # Copy isn't needed lol
# labels is a dataframe containing the corresponding results that we try to predit using the NN
labels = dataset_temp[['Result']]

In [4]:
class NeuralNetwork():
    
    def __init__(self):
        # Seed the random number generator
        np.random.seed(1)
        self.lr = 0.02
        # Set synaptic weights to a 9x1 matrix,
        # with values from -1 to 1 and mean 0
        self.input_hidden_weights = 2 * np.random.random((9, 32)) - 1
        self.output_hidden_weights = 2 * np.random.random((32,1)) - 1

    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return self.sigmoid(x)*(1-self.sigmoid(x))
    
    def tanh(self, x):
        return ((np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x)))
    
    def tanh_derivative(self, x):
        return (1-(self.tanh(x))**2)

    def train(self, X, Y, epochs):
        for epoch in range(epochs):
            # Pass training set through the neural network
            # output = self.think(X)
            # X = X.astype(float)
            h = np.dot(X,self.input_hidden_weights)
            Ah = self.tanh(h)
            h2 = np.dot(Ah,self.output_hidden_weights)
            Yhat = self.sigmoid(h2)
            # Calculate the error rate (MSE DERIVATIVE)
            error = np.mean(np.square(Y-Yhat))
            #print(epoch,error)
            grad_output_hidden_weights = np.dot(Ah.T,(Y-Yhat)*self.sigmoid_derivative(h2))
            grad_input_hidden_weights = np.dot(X.T,np.dot((Y-Yhat)*self.sigmoid_derivative(h2),self.output_hidden_weights.T)*self.tanh_derivative(h))
            self.output_hidden_weights += self.lr * grad_output_hidden_weights
            self.input_hidden_weights += self.lr * grad_input_hidden_weights
            # Multiply error by input and gradient of the sigmoid function
            # Less confident weights are adjusted more through the nature of the function
            # adjustments = np.dot(training_inputs.T, error * self.sigmoid_derivative(output))

            # Adjust synaptic weights
            # self.synaptic_weights += adjustments

    def predict(self, X):
        """
        Pass inputs through the neural network to get output
        """
        # X = X.astype(float)
        h = np.dot(X,self.input_hidden_weights)
        Ah = self.sigmoid(h)
        h2 = np.dot(Ah,self.output_hidden_weights)
        Yhat = self.sigmoid(h2)
        return Yhat


In [5]:
if __name__ == "__main__":

    # Initialize the single neuron neural network
    neural_network = NeuralNetwork()

    # print("Random starting synaptic weights: ")
    # print(neural_network.input_hidden_weights)
    # print(neural_network.output_hidden_weights)
    # The training set, with 4 examples consisting of 3
    # input values and 1 output value
    X = np.array(features, dtype=np.float128)
    Y = np.array(labels, dtype=np.float128)    
    print(X.shape, Y.shape)
    print("Starting Training")
    # split into train test sets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)
    # Train the neural network
    neural_network.train(X_train, Y_train, 5000)
    # print("Synaptic weights after training: ")
    # print(neural_network.input_hidden_weights)
    # print(neural_network.output_hidden_weights)
    Yhat = neural_network.predict(X_test)
    Y_hat = [1 if i>0.5 else 0 for i in Yhat]
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for i,j in zip(Y_test,Y_hat):
        if i==1 and j==1:
            TP+=1
        elif i==0 and j==0:
            TN+=1
        elif i==1 and j==0:
            FN+=1
        elif i==0 and j==1:
            FP+=1
            
    accuracy=(TP+TN)/(TP+TN+FP+FN)
    print(f"accuracy : {accuracy}")
    cm=[[0,0],[0,0]]
    cm[0][0]=TN
    cm[0][1]=FP
    cm[1][0]=FN
    cm[1][1]=TP
    p= TP/(TP+FP)
    r= TP/(TP+FN)
    f1=(2*p*r)/(p+r)
    print("Confusion Matrix : ")
    print(cm)
    print("\n")
    print(f"Precision : {p}")
    print(f"Recall : {r}")
    print(f"F1 SCORE : {f1}")

(96, 9) (96, 1)
Starting Training
accuracy : 0.8
Confusion Matrix : 
[[0, 4], [0, 16]]


Precision : 0.8
Recall : 1.0
F1 SCORE : 0.888888888888889
